# cat2vec

## Aim of the Notebook
The main idea of this notebook is to develop a snippet of code to transform categorical features into embeddings using the Gensim Word2Vec library. Below is an initial draft of the methods that will be used:

### Method 1: `apply_w2v`
This method takes sentences, a Word2Vec model, and the number of features as input. It returns the average word vectors for each sentence.

```python
def apply_w2v(sentences, model, num_features):
    # ...
```

### Method 2: `gen_cat2vec_sentences`
This method takes a DataFrame and returns a list of sentences, where each sentence is a list of categories.

```python
def gen_cat2vec_sentences(df):
    # ...
```

### Method 3: `fit_cat2vec_model`
This method takes a DataFrame, the number of features, and the window size for the Word2Vec model. It returns a trained Word2Vec model.

```python
def fit_cat2vec_model(df, n_cat2vec_features, n_cat2vec_window):
    # ...
```

We will load a toy classification dataset and create an example using a neural network model.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the Iris dataset
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Split the dataset into training and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Show the first few rows of the training data
df_train.head()